In [ ]:
import os
os.remove('snorkel.db')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

### Parsing

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

#### Define Parser

In [ ]:
import os
from snorkel.parser import CorpusParser, HTMLParser, OmniParser
from snorkel.utils import get_ORM_instance
from snorkel.queries import split_corpus

html_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
pdf_path  = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_pdf/'
doc_parser = HTMLParser(path=html_path)
context_parser = OmniParser(pdf_path=pdf_path, session=session)
cp = CorpusParser(doc_parser, context_parser, max_docs=100) 

#### Run Parser

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

session.add(corpus)
session.commit()

#### Backup

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ corpus');

### Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass

Part = candidate_subclass('Part', ['part'])

#### Define Matchers

In [2]:
from snorkel.matchers import RegexMatchSpan, Union

eeca_rgx = ur'([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z]{0,2}[\/]?[A-Z]{0,2}[0-9]?[A-Z]?([(\-|\u2010|\u2011|\u2012|\u2013|\u2014|\u2212)][A-Z0-9]{1,7})?([(\-|\u2010|\u2011|\u2012|\u2013|\u2014|\u2212)][A-Z0-9]{1,2})?)'
eeca_matcher = RegexMatchSpan(rgx=eeca_rgx, longest_match_only=True)
jedec_rgx = '([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)'
jedec_matcher = RegexMatchSpan(rgx=jedec_rgx, longest_match_only=True)
jis_rgx = '(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})'
jis_matcher = RegexMatchSpan(rgx=jis_rgx, longest_match_only=True)
others_rgx = '((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)'
others_matcher = RegexMatchSpan(rgx=others_rgx, longest_match_only=True)
# parts_rgx = '|'.join([eeca_rgx, jedec_rgx, jis_rgx, others_rgx])
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

#### Define ContextSpaces

In [3]:
import os
from collections import defaultdict
from hardware_utils import OmniNgramsPart, get_gold_dict, expand_part_range
from snorkel.utils import ProgressBar
from snorkel.lf_helpers import *
from snorkel.utils import get_ORM_instance
from snorkel.models import Corpus
from snorkel.candidates import OmniNgrams

# # 1 Pass to find all suffixes using a suffix matcher
corpus = get_ORM_instance(Corpus, session, 'Hardware')

eeca_suffix = '^(A|B|C|-16|-25|-40|16|25|40)$'
suffix_matcher = RegexMatchSpan(rgx=eeca_suffix, ignore_case=False)

suffix_ngrams = OmniNgrams(n_max=1)

part_ngrams = OmniNgramsPart(n_max=5) # need to expand it automatically

def extract_dicts(contexts, session):
    suffixes_by_doc = defaultdict(set)
    parts_by_doc = defaultdict(set)
    final_dict = defaultdict(set)
    mapping = [ (u'\u2010', '-'),
                (u'\u2011', '-'),
                (u'\u2012', '-'),
                (u'\u2013', '-'),
                (u'\u2014', '-'),
                (u'\u2212', '-'),
              ]
    pb = ProgressBar(len(contexts))
    for i, context in enumerate(contexts):
        pb.bar(i)
        # Extract Suffixes
        for ts in suffix_matcher.apply(suffix_ngrams.apply(context)):
            row_ngrams = set(get_row_ngrams(ts, infer=True))
            if ('classification' in row_ngrams or 
                'group' in row_ngrams or
                'rank' in row_ngrams or
                'grp.' in row_ngrams):
                suffixes_by_doc[ts.parent.document.name.upper()].add(ts.get_span())
        
        #extract parts
        for ts in parts_matcher.apply(part_ngrams.apply(context)):
            text = ts.get_span()
            for k, v in mapping:
                text = text.replace(k, v)
            if text.endswith('/') or text.endswith('-'):
                continue
                
            if "PNP" in text or "NPN" in text:
                continue
                
            parts_by_doc[ts.parent.document.name.upper()].add(text)
    
    pb.close()
    
    # Process suffixes and parts
    for doc in parts_by_doc.keys():
        for part in parts_by_doc[doc]:
            final_dict[doc].add(part)
            for suffix in suffixes_by_doc[doc]:
                if (suffix == "A" or suffix == "B" or suffix == "C"):
                    if not any(x in part[2:] for x in ['A', 'B', 'C']):
                        final_dict[doc].add(part + suffix)
                else: # if it's 16/25/40
                    if not suffix.startswith('-') and not any(x in part for x in ['16', '25', '40']):
                        final_dict[doc].add(part + '-' + suffix)
                    elif suffix.startswith('-') and not any(x in part for x in ['16', '25', '40']):
                        final_dict[doc].add(part + suffix)

    # TODO: Just return final_set. Temporarily returning the other two for
    # easier debugging.
    return final_dict, suffixes_by_doc, parts_by_doc
            
parts, s, p = extract_dicts(corpus.documents, session)      

[========================================] 100%


In [4]:
import pickle
filename = '1stpass.pkl'
with open(filename, 'w') as f:
    pickle.dump([parts, s, p], f)

In [ ]:
import pickle
filename = '1stpass.pkl'
with open(filename, 'r') as f:
    [parts, s, p] = pickle.load(f)

In [6]:
from pprint import pprint
from hardware_utils import OmniNgramsPart, get_gold_dict, expand_part_range  
pprint(sorted([part for part in parts['BC546']]))

print(sorted([x for x in expand_part_range(u"BC337–16/BC338–16")]))

[u'BC546',
 u'BC546A',
 u'BC546ABU',
 u'BC546ATA',
 u'BC546B',
 u'BC546BTA',
 u'BC546BTF',
 u'BC546C',
 u'BC546CTA',
 u'BC547',
 u'BC547A',
 u'BC547ATA',
 u'BC547B',
 u'BC547BBU',
 u'BC547BTA',
 u'BC547BTF',
 u'BC547C',
 u'BC547CBU',
 u'BC547CTA',
 u'BC547CTFR',
 u'BC548',
 u'BC548A',
 u'BC548B',
 u'BC548BTA',
 u'BC548BU',
 u'BC548C',
 u'BC548CTA',
 u'BC549',
 u'BC549A',
 u'BC549B',
 u'BC549BTA',
 u'BC549BTF',
 u'BC549C',
 u'BC549CTA',
 u'BC550',
 u'BC550A',
 u'BC550B',
 u'BC550C',
 u'BC550CBU',
 u'BC550CTA',
 u'BC556',
 u'BC556A',
 u'BC556B',
 u'BC556C',
 u'BC557',
 u'BC557A',
 u'BC557B',
 u'BC557C',
 u'BC558',
 u'BC558A',
 u'BC558B',
 u'BC558C',
 u'BC559',
 u'BC559A',
 u'BC559B',
 u'BC559C',
 u'BC560',
 u'BC560A',
 u'BC560B',
 u'BC560C']
[u'16', u'BC337\u201316', u'BC337\u201316/BC338\u201316', u'BC338', u'BC338\u201316']


In [ ]:
# Enhance CandidateExtractor to add these suffixes to the parts found in the parts_matcher
from hardware_utils import get_gold_dict
from collections import defaultdict

# Make parts list
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'

# # Gold parts looks like this:
#     # set([('112823', 'BC546'),
#     #      ('112823', 'BC546B'),
#     #      ('112823', 'BC546BG'),            if ts.parent.document.name.upper() == "MOTOS03189-1":
#     #      ('112823', 'BC546BRL1'),
#     #      ('112823', 'BC546BRL1G'),
#     #      ('112823', 'BC546BZL1G'),
#     #      ('112823', 'BC547'),
#     #      ('112823', 'BC547A'),
#     #      ('112823', 'BC547ARL'),
#     #      ('112823', 'BC547ARLG'),
#     #      ('112823', 'BC547AZL1G'),
#     #      ('112823', 'BC547B'),
#     #      ('112823', 'BC547BG'),
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)

gold = set()
gold_parts_by_doc = defaultdict(set)
for part in gold_parts:
    gold_parts_by_doc[part[0]].add(part[1])
    gold.add((part[0], part[1]))
print len(gold)

homemade = set()
for doc, parts in parts.iteritems():
    for part in parts:
        homemade.add((doc,part))
print len(homemade)

In [ ]:
tp = gold.intersection(homemade)
fp = homemade - gold
fn = gold - homemade
print "TP: ", len(tp)
print "FP: ", len(fp)
print "FN: ", len(fn)
print "TOTAL: ", len(tp) + len(fp) + len(fn)

In [ ]:
from pprint import pprint
pprint(fp)

In [7]:
from hardware_utils import OmniNgramsPart

part_ngrams = OmniNgramsPart(parts_by_doc=parts, n_max=5)

#### Run CandidateExtractor

In [8]:
from snorkel.models import Corpus
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance


ce = CandidateExtractor(Part, 
                        [part_ngrams], 
                        [parts_matcher])

for corpus_name in ['Hardware']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

Extracting Candidates from Corpus (Hardware)
[========================================] 100%
CPU times: user 1min 8s, sys: 584 ms, total: 1min 8s
Wall time: 1min 8s
Candidate Set (Hardware Candidates) contains 23841 Candidates


### Assess Recall

Using the dictionary approach we have

- **24820** candidates for part numbers
- 811 entity-level candidates and **100% recall**.

In [9]:
from hardware_utils import entity_level_total_recall, most_common_document, get_gold_dict
from snorkel.utils import get_ORM_instance
from snorkel.models import Candidate, Corpus

all_candidates = session.query(Candidate).all()
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'

corpus = get_ORM_instance(Corpus, session, 'Hardware')
(tp, fp, fn) = entity_level_total_recall(
    all_candidates, gold_file, None, corpus=corpus, relation=False)
print "TP: ", len(tp)
print "FP: ", len(fp)
print "FN: ", len(fn)
print "TOTAL: ", len(tp) + len(fp) + len(fn)

Preparing candidates...
[========================================] 100%
Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 1732 
Entity-level Gold: 809
Intersection Candidates: 806
----------------------------------------
Overlap with Gold:  0.9963

TP:  806
FP:  926
FN:  3
TOTAL:  1735


In [10]:
from pprint import pprint
fps = list(fp)
pprint(sorted(fps)[:100])

[(u'112823', u'BC546/D'),
 (u'112823', u'BC548A'),
 (u'2N3906-D', u'1N916'),
 (u'2N3906-D', u'1N916C'),
 (u'2N3906-D', u'1N916CS'),
 (u'2N3906-D', u'2N390D'),
 (u'2N4123-D', u'2N412D'),
 (u'2N4124', u'1N916'),
 (u'2N6426-D', u'2N6426IS'),
 (u'2N6426-D', u'2N6426ISA'),
 (u'2N6426-D', u'2N642D'),
 (u'AUKCS04635-1', u'2N3904'),
 (u'BC182-D', u'BC182/D'),
 (u'BC337', u'BC327'),
 (u'BC337', u'BC327-16'),
 (u'BC337', u'BC327-25'),
 (u'BC337', u'BC327-40'),
 (u'BC337', u'BC328'),
 (u'BC337', u'BC328-16'),
 (u'BC337', u'BC328-25'),
 (u'BC337', u'BC328-40'),
 (u'BC337', u'BC33716'),
 (u'BC337', u'BC33725'),
 (u'BC337', u'BC33740'),
 (u'BC337', u'BC33740BU'),
 (u'BC337', u'BC338-16'),
 (u'BC337', u'BC338-40'),
 (u'BC337', u'BC33825'),
 (u'BC337-D', u'BC337-25BC337'),
 (u'BC337-D', u'BC337-XX'),
 (u'BC337-D', u'BC337/D'),
 (u'BC546', u'BC548A'),
 (u'BC546', u'BC549A'),
 (u'BC546', u'BC550A'),
 (u'BC546', u'BC550B'),
 (u'BC546', u'BC556'),
 (u'BC546', u'BC556A'),
 (u'BC546', u'BC556B'),
 (u'BC546'

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ candidates');